In [ ]:
import os.path as op
import os
import shutil as sh
from ruamel.yaml import YAML
import nbformat as nbf
from subprocess import run, PIPE

from glob import glob

## Inputs

# Path to ipynb file
ipynb_file = './jupyter_book/tests/site/content/tests/report.ipynb'

folder = 'TMP_book/'

import os
tmp_folder = './TMP'
path_out = op.join(tmp_folder, 'content', op.basename(ipynb_file))
os.makedirs(op.join(tmp_folder, 'content'), exist_ok=True)
sh.copy2(ipynb_file, path_out)

## Hide code cells

ntbk = nbf.read(path_out, nbf.NO_CONVERT)

for cell in ntbk['cells']:
    if cell['cell_type'] == 'code':
        tags = cell['metadata'].get('tags', [])
        if not 'hidecode' in tags:
            tags.append('hidecode')
        cell['metadata']['tags'] = tags

nbf.write(ntbk, path_out)
    

## Build the book

yaml = YAML()
config = {'show_sidebar': False, "baseurl": "", "url": ""}
with open(op.join(tmp_folder, 'toc.yml'), 'w') as ff:
    yaml.dump(config, ff)

cmd = "jupyter-book create TMP_book --content-folder ./TMP/content --overwrite --config ./TMP/toc.yml"
out = run(cmd.split(), stderr=PIPE, stdout=PIPE)



cmd = "jupyter-book build ."
out = run(cmd.split(), stderr=PIPE, stdout=PIPE, cwd=folder)
print(out.stderr.decode())

cmd = 'bundle exec jekyll build -d ../TMP_HTML'
err = run(cmd.split(), cwd=folder, stderr=PIPE, stdout=PIPE)

print(err.stderr.decode())

# Remove paths relative to root of folder

HTML_files = glob(op.join('TMP_HTML', '*.html'))

for ifile in HTML_files:
    with open(ifile, 'r') as ff:
        text = ff.read()
        text = text.replace('href="/', 'href="')
        text = text.replace('src="/', 'src="')
    with open(ifile, 'w') as ff:
        ff.write(text)
        

!rm -rf /home/choldgraf/Dropbox/TMP_HTML && cp -r TMP_HTML /home/choldgraf/Dropbox/TMP_HTML

